In [12]:
# Created by Balakrishna Vagvala and Sherrill Kirk
# This code depends upon the packages and libraries listed in the next cell
# Date of creation: 12/1/2020
# Script purpose: this script processes the Test America Lab delivered data, compares with expected fields file and make ready to be uploaded to database.

In [13]:
# We use openpyxl to load excel work book, this will help in having our current data as new sheet in the workbook
# we use shutil to copy a file from it's location to a desired location,
# this will help in copy pasting our original file to make changes on it
# We use difflib library and import sequencematcher function, this will help us in comparing strings and give us a match ratio
# We use numpy library and import it as np to make numerical operations for our program
# we use pandas library and immport it as np to make data operations for our lab data
# we use time library to import curent time of our function
# we use OS library for different functions such as writing data to file and saving files
# We use glob library and its glob function finds all the pathnames matching a specified pattern
# We use xlsx writer to write sheets and reed sheets into an excel file, to install for the first time, uncomment the below line
#!pip install XlsxWriter

In [14]:
#importing pandas for data operations, numpy for numerical operations. time is used to have present time.
import pandas as pd
import numpy as np
import time
#sequence matcher is used to compare strings and openpyxl is used to write workbooks.
from difflib import SequenceMatcher
from openpyxl import load_workbook
from shutil import copyfile
    
from re import search
#importing reading functions and declaring the location of our file.
import os, glob
import os.path
import xlsxwriter

In [15]:
########################
#Variables to be defined
########################

#Expected field names definition excel file:
crosswalkpath = r'C:\ROMN\Monitoring\Streams\Data\DataGathering\Lab\TestAmerica\TestAmerica_Delivery_ExpectedFields.xlsx'

#Output Directory:
currentoutputpath= r'C:\ROMN\Monitoring\Streams\Data\DataGathering\Lab\TestAmerica\2021\Preprocessed'

#Directory with the Test America lab deliverable .csv files to be processed:
path =r'C:\ROMN\Monitoring\Streams\Data\DataGathering\Lab\TestAmerica\2021'

#Wild Card Syntax used to define the Test America lab deliverable files to be processed 
wildCardSyntax = "*TalStandard*.csv"

#Output file name suffix for the Logfile and Subset/Appended file post processing
outPutFileSuffix = "TestAmericaAll_2021"


In [16]:
crosswalk = pd.read_excel(crosswalkpath)
checkfile = crosswalk

#Path where all the excel files are residing to be processed
all_files = glob.glob(os.path.join(path, wildCardSyntax))

# this code block saves our files into a path we desire
processedsavefiles = currentoutputpath
processed_path = (currentoutputpath+'\\')

# runtime for file

timestr = time.strftime("_%Y%m%d_%H%M")
runtimenow = timestr

# the opening the log file
logfile = open(currentoutputpath+"\\" + outPutFileSuffix +runtimenow+".txt","w+")


listofdataframes = []
listofunchangeddataframes =[]

for  no,f in enumerate(all_files):
    
    #df = pd.read_excel(f)
    df = pd.read_csv(f)
#     uncomment below line if its csv file, remove the '#' symbol
#    df = pd.read_csv(f)
    listofunchangeddataframes.append(df)
    

    
    #file to process , this is an excel file's path
    filename= os.path.basename(f)
    print(filename)
    sep = '.'
    filename = filename.split(sep, 1)[0]

    

    firstline=[]
    firstline.append("File Being Processed is: " + filename )
    #toread the lab file, we mention sheet name of excel file in the second argument, here GRKO Data is our sheet name in excel file by lab

    testamfields = list(df.columns)
    expected_fields = checkfile['ExpectedFields'].tolist() 
    
    matchingfields =[]
    lenghtoffields = []
    morethanninety = []
    lessthanninety = []
    newfields=[]
    duplicatefields=[]
    missedfields = []
    x=''
    if len(testamfields)>len(expected_fields):
        lenghtoffields.append('\n\n#########File: '+filename+' file has new columns.\n')

    if len(testamfields)<len(expected_fields):
        lenghtoffields.append('\n\n#########File: '+filename+' file has less columns than expected.\n')

    duplicatefields.append ('\n\n#####\n\nFields with duplicate field names:\n') 
    missedfields.append ('\n\n######\n\nFields which are expected, but not in our data:\n')
    matchingfields.append ('\n\n#####\n\nFields that matched as expected:\n') 
    morethanninety.append ('\n\n#####\n\nFields without 100% match but had more than 90% Match:\n') 
    lessthanninety.append ('\n\n#####\n\nFields with less than 90% match but had more than 80% Match:\n') 
    newfields.append ('\n\n#####\n\nNew Fields, these are not found in crosswalk and have been removed from data:\n') 

    
    for i in expected_fields: 
        for j in testamfields:
            k = i+"."+"1"
            if search(k, j):
                print(k)
                duplicatefields.append("Warning: "+i+ " has a duplicate field \n")
                break
    
#     for i in testamfields:   
#         for j in expected_fields:
#             if j in i:
#                 if j != i:
#                     ratio = SequenceMatcher(None,i,j).ratio()
#                     if ratio != 1:
#                         duplicatefields.append("Warning: "+i+ " is a duplicate field, almost matched with "+j+" \n")                    
#                     x = testamfields.index(i)
#                     cols = [i for i in range(df.shape[1])]
#                     cols.remove(x)
#                     cols.remove(x+1)
#                     df = df.iloc[:,cols]
#                     del testamfields[x]
#                     del testamfields[x]

    for i in testamfields:     
        if i in expected_fields:
            matchingfields.append("column: "+ i +" has successfully matched \n")
            continue      

        else:
            result = ""
            count = 0
            ratio = 0  

            for words in expected_fields:
                ratio = SequenceMatcher(None,i,words).ratio()
                if ratio > count:
                    count = ratio
                    result =words         

            if count > 0.9:
                x = testamfields.index(i)
                y = expected_fields.index(result)        
                morethanninety.append("Warning: "+testamfields[x]+" was crosswalk to "+ result + " has matched with more than 90% accuracy \n")
                testamfields[x] = expected_fields[y]
                
            elif count> 0.8:
                x = testamfields.index(i)
                y = expected_fields.index(result)        
                morethanninety.append("Warning: "+testamfields[x]+" was crosswalk to "+ result + " has matched with more than 90% accuracy \n")
                testamfields[x] = expected_fields[y]



    # temp to check the missing columns in our processed lab deliverable 
    missedfields = []
    missedfields.append ('\n\n######\n\nFields which are expected, but not in our data:\n')
    temp =  expected_fields.copy()
    for i in testamfields:
        if i in temp:
            temp.remove(i) 
    for i in temp:
        missedfields.append("column: "+ i + " is expected but is not in our current data\n ")        

    df.columns = testamfields

    for i in testamfields:
        if i not in expected_fields:
            newfields.append("Warning: "+i+" is a new field, look into it\n")
            x = list(df.columns).index(i)
            del testamfields[x]
            cols = [i for i in range(df.shape[1])]
            cols.remove(x)
            df = df.iloc[:,cols]


    df.columns = testamfields
    # to reindex columns based on expected fields

    df = df.loc[:,~df.columns.duplicated()]

    df = df.reindex(expected_fields, axis=1)
    # this will drop columns with no values which are picked up from re-indexing
#     df = df.dropna(axis=1)

    worked_files = firstline  + lenghtoffields+ morethanninety +lessthanninety + duplicatefields + missedfields+newfields + matchingfields
    for i in range(len(worked_files)):
        logfile.write(worked_files[i])

    logfile.write('\n\n######\nthis is end of the Log File for '+filename+'\n\n###########\n\n\n\n\n\n\n\n')
          

#     pd.set_option("display.max_rows", None, "display.max_columns", None)
#     df

    #saving our new csv file into a downloadable format. and it can be saved
    # df.to_csv( "data_fieldsprocessed"+timestr+".csv", index=False, encoding='utf-8-sig')
    #if you want an excel output
    # f = open
    # df.to_excel("pre-processed"+timestr+".xls", index=False, encoding='utf-8-sig')

    # to make a duplicate of our file and export our processed 
    # copyfile ('D:\Bala\Testamerica\Testamerica_practice\Test america Practice data 28-11\280-136365-1_TalStandard.csv\' , './processed'+filename+'.xlsx')

#     data

    # rename our copied file and place it at the location we want

    # os.rename(r'D:\Bala\Testamerica\output\processed.xlsx', r'D:\Bala\Testamerica\output\Testamerica_processed_' + filename + '.xlsx')

    #Replacing Nulls with missing values for easae of operation for the sample type desc coloumn.
    df['Sample Type Desc'] = df['Sample Type Desc'].fillna('Missing value')
    
    #removing description types of Lab control sample, Lab control sample duplicate and method blank from Sample type description of Test america data
    df_new = df.loc[(df['Sample Type Desc'] != 'Lab Control Sample') & (df['Sample Type Desc'] != 'Lab Control Sample Duplicate')& (df['Sample Type Desc'] != 'Method Blank' )]
    #df_new is the subset data set.
    df = df_new
#     df = df_new.replace('Missing value', '')
    df.loc[df['Sample Type Desc'] == 'Missing value', 'Sample Type Desc'] = ''
    df_new = df
    listofdataframes.append(df_new)
    
logfile.close() 

280-147837-1_TalStandard.csv
280-148304-1_TalStandard.csv
280-149036-1_TalStandard.csv
280-149967-1_TalStandard.csv
280-150396-1_TalStandard.csv
280-151067-1_TalStandard.csv
280-152639-1_TalStandard.csv
280-154540-1_TalStandard.csv
280-156396-1_TalStandard.csv


In [17]:
x = listofdataframes[1]
display(x)

,Lab Sample ID,Client Sample ID,Matrix,Sample Type,Collection Date,Percent Moisture,Analysis Method,Dilution Factor,Analysis Date,CAS,Analyte,Result,Unit,Flag,High Limit,High Limit Type,Low Limit,Low Limit Type,Percent Recovery,Lower Recovery Limit,Upper Recovery Limit,RPD,RPD Limit,Receipt Lab,Project Description,SDG,Lab Job ID,COC ID,Sample Type Desc,Receive Date,Total Sample Amount,Total Sample Amount Unit,Leach Batch,Leach Method,Leach Date,Prep Batch,Prep Method,Prep Date,Prep Type,Initial Amount,Initial Amount Unit,Final Amount,Final Amount Unit,Re-Analysis Type,Analysis Batch,Analysis Lab,Instrument ID,Column/Detector ID,Basis,Analyte Type,Result Status,TPU,TPU Sigma,Decision Level,Retention Time,Spike Amount,Expected Amount,RER,RER Limit,Lower Breach Limit,Upper Breach Limit
0,280-148304-2,21NPS0000684 E3.WAC,Water,NaN,05/04/2021 08:50,NaN,6020A,1,05/14/2021 09:12,7440-38-2,Arsenic,14,ug/L,NaN,5.0,RL,0.330,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-148304-1,NaN,,05/06/2021 09:45,NaN,NaN,NaN,NaN,NaN,280-535860,3020A,05/12/2021 15:45,Total/NA,50.000,mL,50.0,mL,NaN,280-536175,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,280-148304-2,21NPS0000684 E3.WAC,Water,NaN,05/04/2021 08:50,NaN,6020A,1,05/14/2021 09:12,7440-39-3,Barium,59,ug/L,NaN,1.0,RL,0.290,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-148304-1,NaN,,05/06/2021 09:45,NaN,NaN,NaN,NaN,NaN,280-535860,3020A,05/12/2021 15:45,Total/NA,50.000,mL,50.0,mL,NaN,280-536175,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,280-148304-2,21NPS0000684 E3.WAC,Water,NaN,05/04/2021 08:50,NaN,6020A,1,05/14/2021 09:12,7440-43-9,Cadmium,0.28,ug/L,J,1.0,RL,0.270,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-148304-1,NaN,,05/06/2021 09:45,NaN,NaN,NaN,NaN,NaN,280-535860,3020A,05/12/2021 15:45,Total/NA,50.000,mL,50.0,mL,NaN,280-536175,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,280-148304-2,21NPS0000684 E3.WAC,Water,NaN,05/04/2021 08:50,NaN,6020A,1,05/14/2021 09:12,7440-50-8,Copper,42,ug/L,NaN,2.0,RL,0.560,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-148304-1,NaN,,05/06/2021 09:45,NaN,NaN,NaN,NaN,NaN,280-535860,3020A,05/12/2021 15:45,Total/NA,50.000,mL,50.0,mL,NaN,280-536175,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,280-148304-2,21NPS0000684 E3.WAC,Water,NaN,05/04/2021 08:50,NaN,6020A,1,05/14/2021 09:12,7439-92-1,Lead,5.1,ug/L,NaN,1.0,RL,0.180,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-148304-1,NaN,,05/06/2021 09:45,NaN,NaN,NaN,NaN,NaN,280-535860,3020A,05/12/2021 15:45,Total/NA,50.000,mL,50.0,mL,NaN,280-536175,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,280-148304-2,21NPS0000684 E3.WAC,Water,NaN,05/04/2021 08:50,NaN,6020A,1,05/14/2021 09:12,7782-49-2,Selenium,0.44,ug/L,J,5.0,RL,0.370,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-148304-1,NaN,,05/06/2021 09:45,NaN,NaN,NaN,NaN,NaN,280-535860,3020A,05/12/2021 15:45,Total/NA,50.000,mL,50.0,mL,NaN,280-536175,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,280-148304-2,21NPS0000684 E3.WAC,Water,NaN,05/04/2021 08:50,NaN,6020A,1,05/14/2021 09:12,7440-22-4,Silver,0.19,ug/L,J,5.0,RL,0.033,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-148304-1,NaN,,05/06/2021 09:45,NaN,NaN,NaN,NaN,NaN,280-535860,3020A,05/12/2021 15:45,Total/NA,50.000,mL,50.0,mL,NaN,280-536175,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,280-148304-2,21NPS0000684 E3.WAC,Water,NaN,05/04/2021 08:50,NaN,6020A,1,05/14/2021 09:12,7440-66-6,Zinc,37,

In [18]:
 

count = 0

for  no,f in enumerate(all_files):
    for i in range(len(listofdataframes)):          
        if i == count:
            count = count + 1
            filename= os.path.basename(f)
            print(filename)
            sep = '.'
            filename = filename.split(sep, 1)[0]

            processed_path = (processedsavefiles+ "\\" + filename + '_processed.xlsx')

            #read_file = pd.read_excel(f)
            read_file = pd.read_csv(f)
    #         read_file = pd.read_csv (f)
            read_file.to_excel (processed_path,sheet_name = 'OriginalData', index = None, header=True)

            book = load_workbook(processed_path)
            writer = pd.ExcelWriter(processed_path, engine = 'openpyxl')
            writer.book = book
            # enter the desired sheet name below
            sheetname = 'preprocessed' #enter your sheet name here in the space, we can have it as a prefix as you want
            z = listofdataframes[i]
            z.to_excel(writer, sheet_name = sheetname, index = None)

            writer.save()
            writer.close()
            break
            break
        else:
            continue
            

280-147837-1_TalStandard.csv
280-148304-1_TalStandard.csv
280-149036-1_TalStandard.csv
280-149967-1_TalStandard.csv
280-150396-1_TalStandard.csv
280-151067-1_TalStandard.csv
280-152639-1_TalStandard.csv
280-154540-1_TalStandard.csv
280-156396-1_TalStandard.csv


In [19]:
#combining all csv files into a single file
combined_csv = pd.concat( listofdataframes, axis=0, ignore_index=True)
df = combined_csv

In [20]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

,Lab Sample ID,Client Sample ID,Matrix,Sample Type,Collection Date,Percent Moisture,Analysis Method,Dilution Factor,Analysis Date,CAS,Analyte,Result,Unit,Flag,High Limit,High Limit Type,Low Limit,Low Limit Type,Percent Recovery,Lower Recovery Limit,Upper Recovery Limit,RPD,RPD Limit,Receipt Lab,Project Description,SDG,Lab Job ID,COC ID,Sample Type Desc,Receive Date,Total Sample Amount,Total Sample Amount Unit,Leach Batch,Leach Method,Leach Date,Prep Batch,Prep Method,Prep Date,Prep Type,Initial Amount,Initial Amount Unit,Final Amount,Final Amount Unit,Re-Analysis Type,Analysis Batch,Analysis Lab,Instrument ID,Column/Detector ID,Basis,Analyte Type,Result Status,TPU,TPU Sigma,Decision Level,Retention Time,Spike Amount,Expected Amount,RER,RER Limit,Lower Breach Limit,Upper Breach Limit
0,280-147837-2,20NPS0000657 E3.WAC,Water,NaN,04/16/2021 12:45,NaN,6020A,1,05/10/2021 17:19,7440-38-2,Arsenic,7.9,ug/L,NaN,5.0,RL,0.330,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-147837-1,NaN,,04/23/2021 10:25,NaN,NaN,NaN,NaN,NaN,280-535286,3020A,05/07/2021 15:36,Total/NA,50.000,mL,50.0,mL,NaN,280-535666,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,280-147837-2,20NPS0000657 E3.WAC,Water,NaN,04/16/2021 12:45,NaN,6020A,1,05/10/2021 17:19,7440-39-3,Barium,50,ug/L,NaN,1.0,RL,0.290,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-147837-1,NaN,,04/23/2021 10:25,NaN,NaN,NaN,NaN,NaN,280-535286,3020A,05/07/2021 15:36,Total/NA,50.000,mL,50.0,mL,NaN,280-535666,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,280-147837-2,20NPS0000657 E3.WAC,Water,NaN,04/16/2021 12:45,NaN,6020A,1,05/10/2021 17:19,7440-43-9,Cadmium,ND,ug/L,NaN,1.0,RL,0.270,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-147837-1,NaN,,04/23/2021 10:25,NaN,NaN,NaN,NaN,NaN,280-535286,3020A,05/07/2021 15:36,Total/NA,50.000,mL,50.0,mL,NaN,280-535666,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,280-147837-2,20NPS0000657 E3.WAC,Water,NaN,04/16/2021 12:45,NaN,6020A,1,05/10/2021 17:19,7440-50-8,Copper,18,ug/L,NaN,2.0,RL,0.560,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-147837-1,NaN,,04/23/2021 10:25,NaN,NaN,NaN,NaN,NaN,280-535286,3020A,05/07/2021 15:36,Total/NA,50.000,mL,50.0,mL,NaN,280-535666,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,280-147837-2,20NPS0000657 E3.WAC,Water,NaN,04/16/2021 12:45,NaN,6020A,1,05/10/2021 17:19,7439-92-1,Lead,2.5,ug/L,B,1.0,RL,0.180,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-147837-1,NaN,,04/23/2021 10:25,NaN,NaN,NaN,NaN,NaN,280-535286,3020A,05/07/2021 15:36,Total/NA,50.000,mL,50.0,mL,NaN,280-535666,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,280-147837-2,20NPS0000657 E3.WAC,Water,NaN,04/16/2021 12:45,NaN,6020A,1,05/10/2021 17:19,7782-49-2,Selenium,ND,ug/L,NaN,5.0,RL,0.370,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-147837-1,NaN,,04/23/2021 10:25,NaN,NaN,NaN,NaN,NaN,280-535286,3020A,05/07/2021 15:36,Total/NA,50.000,mL,50.0,mL,NaN,280-535666,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,280-147837-2,20NPS0000657 E3.WAC,Water,NaN,04/16/2021 12:45,NaN,6020A,1,05/10/2021 17:19,7440-22-4,Silver,ND,ug/L,NaN,5.0,RL,0.033,MDL,NaN,NaN,NaN,NaN,NaN,"Eurofins TestAmerica, Denver",Grant Kohrs Ranch,NaN,280-147837-1,NaN,,04/23/2021 10:25,NaN,NaN,NaN,NaN,NaN,280-535286,3020A,05/07/2021 15:36,Total/NA,50.000,mL,50.0,mL,NaN,280-535666,"Eurofins TestAmerica, Denver",MT_077,NaN,WET,Target,PRIMARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,280-147837-2,20NPS0000657 E3.WAC,Water,NaN,04/16/2021 12:45,NaN,6020A,1,05/10/2021 17:19,7440-66-6,Zinc,21,u

In [21]:
#checking our subset data numbers
df['Sample Type Desc'].value_counts()

                          646
Matrix Spike              172
Matrix Spike Duplicate    172
Name: Sample Type Desc, dtype: int64

In [22]:
#to download our subset data and rename our subset to df.
df.to_csv( currentoutputpath + "\\" + outPutFileSuffix + timestr+".csv", index=False, encoding='utf-8-sig')